In [ ]:
import os
from util import data
from time import time
from pdb import set_trace
from multiprocessing import Pool

%load_ext autoreload
%autoreload 2

default_pool_size = max(1, os.cpu_count())


In [2]:
scores = data.load_data()

Serialized scores found, loading...
Scores loaded in 24.31 seconds.


In [3]:
dataset = data.HaydnDataset(data=scores)
print("There are {} items in the dataset.".format(len(dataset)))

Building dataset...
Finished building dataset in 7.22 seconds.
There are 99 items in the dataset.


In [4]:
# get all of the datasets and filter out the empty ones
start = time()
states = list(filter(lambda ds: ds is not None, dataset))
total_ticks = sum(map(lambda state: state.shape[2], states)) * states[0].shape[0]
print("There are {} final corpi with a total of {} ticks. Loaded in {:.2f} seconds.".format(len(states), total_ticks, time()-start))

There are 81 final corpi with a total of 1872988 ticks. Loaded in 24.78 seconds.


In [5]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # useful stateless functions
import numpy as np
from torch.utils import data

In [6]:
class Center_Model(nn.Module):
    def __init__(self, center_input_size, center_output_size):
        super(Center_Model, self).__init__()
        
        self.center_affine = nn.Linear(center_input_size, center_output_size)
#         self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, center_input):
        
        center_out = self.center_affine(center_input)
#         output = self.relu(center_out)
        output = self.softmax(center_out)
        
        return output
  

In [7]:
class Result_Model(nn.Module):
    def __init__(self, merged_input_size, output_size):
        super(Result_Model, self).__init__()
        
        self.affine = nn.Linear(merged_input_size, output_size)
#         self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, merged_input):
        
        out = self.affine(merged_input)
#         output = self.relu(out)
        output = self.softmax(out)
        
        return output

In [8]:
class Left_Model(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size, output_size):
        super(Left_Model, self).__init__()
        
        self.left_lstm = nn.LSTM(lstm_input_size, lstm_hidden_size)
        self.affine = nn.Linear(lstm_hidden_size, output_size)
#         self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, left_seq):
        '''
        input_size = data_dim
        LSTM input shape: [sqd_len, batch=1, input_size]
        1 = num_layers * num_directions
        LSTM hn output shape: [1, batch, hidden_size]
        LSTM output shape: [seq_len, batch, num_directions * hidden_size]
        
        forward LSTM output shape: [right_seq * batch, hidden_size * num_directions]
        
        model forward output shape: [seq_length, output_size]
        '''
        
        left_out, (left_hn, left_cn) = self.left_lstm(left_seq)
        seq_len, batch, size = left_out.shape
        left_results  = left_out.view(seq_len * batch, size)
        
        out = self.affine(left_results)
#         output = self.relu(out)
        output = self.softmax(out)
        
        return output, left_hn

In [9]:
class Right_Model(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size, output_size):
        super(Right_Model, self).__init__()
        
        self.right_lstm = nn.LSTM(lstm_input_size, lstm_hidden_size)
        self.affine = nn.Linear(lstm_hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax()
        
    def forward(self, right_seq):
        '''
        input_size = data_dim
        LSTM input shape: [sqd_len, batch=1, input_size]
        1 = num_layers * num_directions
        LSTM hn  shape: [1, batch, hidden_size]
        LSTM output shape: [seq_len, batch, num_directions * hidden_size]
        
        forward LSTM output shape: [right_seq * batch, hidden_size * num_directions]
        
        model forward output shape: [seq_length, output_size]
        '''
        right_out, (right_hn, right_cn) = self.right_lstm(right_seq)
        
        seq_len, batch, size = right_out.shape
        right_results  = right_out.view(seq_len * batch, size)
        
        out = self.affine(right_results)
#         output = self.relu(out)
        output = self.softmax(out)
        
        return output, right_hn

In [10]:
class Arti_LSTM_Model(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size, output_size):
        super(Arti_LSTM_Model, self).__init__()
        
        self.left_articulate = nn.LSTM(lstm_input_size, lstm_hidden_size)
        self.affine = nn.Linear(lstm_hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, left_seq):
        '''
        input_size = data_dim
        LSTM input shape: [sqd_len, batch=1, input_size]
        1 = num_layers * num_directions
        LSTM hn output shape: [1, batch, hidden_size]
        LSTM output shape: [seq_len, batch, num_directions * hidden_size]
        
        forward LSTM output shape: [right_seq * batch, hidden_size * num_directions]
        
        model forward output shape: [seq_length, 1]
        '''
        
        left_out, (left_hn, left_cn) = self.left_articulate(left_seq)
        seq_len, batch, size = left_out.shape
        left_results  = left_out.view(seq_len * batch, size)
        
        out = self.affine(left_results)
        output = self.sigmoid(out)
        
        return output, left_hn

In [11]:
def train(left_model, right_model, center_model, result_model, arti_model,
          left_tensor, right_tensor, center_tensor, 
          center_target_tensor, arti_target_tensor,
          opt_affine, opt_arti):        
    
    left_output, left_hn = left_model(left_tensor)
    right_output, right_hn = right_model(right_tensor)
    center_output = center_model(center_tensor)
    arti_output, arti_hn = arti_model(left_tensor)
    
#     left_loss = F.nll_loss(left_output, left_target_tensor)
#     opt_left.zero_grad()
#     left_loss.backward(retain_graph=True)
#     opt_left.step()
    
#     right_loss = F.nll_loss(right_output, right_targte_tensor)
#     opt_right.zero_grad()
#     right_loss.backward(retain_graph=True)
#     opt_right.step()
    
#     center_loss = F.nll_loss(center_output, center_target_tensor)
#     opt_center.zero_grad()
#     center_loss.backward(retain_graph=True)
#     opt_center.step()
    
    arti_loss = F.l1_loss(arti_output, arti_target_tensor)
    opt_arti.zero_grad()
    arti_loss.backward()
    opt_arti.step()
    
    '''
    Get the last predicted output from LSTM and reshape to column vector
    '''
    left_out_seq_final = left_output[-1, :].view(1, -1)
    right_out_seq_final = right_output[-1, :].view(1, -1)
#     print('left_out_seq_final shape: {x}'.format(x=left_out_seq_final.shape))
#     print('center_output shape: {x}'.format(x=center_output.shape))
#     print('right_out_seq_final shape: {x}'.format(x=right_out_seq_final.shape))
    comobined_result = torch.cat((left_out_seq_final, center_output, right_out_seq_final), 1)
    result_output = result_model(comobined_result)
 
    result_loss = F.nll_loss(result_output, center_target_tensor)
    opt_affine.zero_grad()
    result_loss.backward(retain_graph=True)
    opt_affine.step()
        
    return result_output, result_loss.item(), arti_loss.item()

In [12]:
def findNoneZeroIndex(m):
    '''
    m is a 2 dimentional numpy array
    
    m is modified that the last entry is deleted. 
    
    return an array with shape [k, ] contains the (only) nonzero index of each row in m
    k = the number of rows in m
    '''
    x, y = m.shape
#     print('m shape: {x}'.format(x=m.shape))
    n = np.zeros(x)
    for i in range (x):
        m_row = m[i, :]
        m_row_clipped = np.delete(m_row, -1)
#         print('m_row_clipped shape: {x}'.format(x=m_row_clipped.shape))
#         print(m_row_clipped)
        index = np.nonzero(m_row_clipped)[0]
#         print('idx: {x}'.format(x=index))
        n[i] = index
    return n

In [13]:
def getTargetIndexTensor(left, right, c_target_tensor):
    center_clip = np.delete(c_target_tensor, -1)
    center_target_index_array = np.nonzero(center_clip)[0]
    center_target_index_tensor = torch.from_numpy(center_target_index_array).long()

    
    left_target = findNoneZeroIndex(left)
    right_target = findNoneZeroIndex(right)
    right_index_target = np.flip(right_target, axis=0)
    
    left_index_target_tensor = torch.from_numpy(left_target).long()
    right_index_target_tensor = torch.from_numpy(right_index_target.copy()).long()
    
    return (left_index_target_tensor, right_index_target_tensor, center_target_index_tensor)

In [14]:
def generateMiniBatchFromOneMusicForPart(music_data, seq_length, part):
    '''    
    input: music_data. A numpy array with shape [4, tick, data_dim]
    return: a list contains miniBatch data for one part. Each item in the miniBatch list contains:
            - left_tensor: shape [seq_length, 1, D]
            - right_tensor: shape [seq_length, 1, D]
            - center_tensor: shape [1, D]
            - left_target_tensor: shape [seq_length, ]
            - right_targte_tensor: shape [seq_length, ]
            - center_target_tensor: shape [1, 1]
            - articulation_target_tensor: shape[1,]

        D = size of array contains the data from three other parts
                
    '''
    miniBatches = []
    
    _, music_length, data_dim = music_data.shape
#     music_length = 6 # this is only for testing
    i = 0 # TODO: maybe add functinality to pad the beginning and end.
    center_inx = i + seq_length
    while (center_inx + seq_length) < music_length:
        left_range_start = center_inx - seq_length
        left_range_end = center_inx 
        center_index = center_inx
        right_range_start = center_inx + 1
        right_range_end = right_range_start + seq_length
        
        left_range = music_data[part, left_range_start:left_range_end, :]
        seq_l, notes_l = left_range.shape
        left_tensor = torch.from_numpy(left_range.reshape(seq_l, 1, notes_l)).float()
        
        right_range = music_data[part, right_range_start:right_range_end, :]
        right_flipped = np.flip(right_range, axis=1).copy()
        seq_l, notes_l = right_flipped.shape
        right_tensor = torch.from_numpy(right_flipped.reshape(seq_l, 1, notes_l)).float()
        
        center_rep = music_data[:, center_inx, :]
        center_other_parts = np.delete(center_rep, part, axis=0)
        part_c, notes_c = center_other_parts.shape
        center_tensor = torch.from_numpy(center_other_parts.reshape(1, part_c * notes_c)).float()
        
        '''
        center_target_range shape: [data_dim,]
        '''
        center_target_range = music_data[part, center_inx+1, :]
        
        '''
        left range and right range are in dimension [seg_length, data_dim]
        '''
        left_target_tensor_range = music_data[part, left_range_start+1:left_range_end+1, :]
        right_target_tensor_range = music_data[part, right_range_start-1:right_range_end-1, :]

        '''
        Get target tensor for left, right, and center note model.
        '''
        left_index_target_tensor, right_index_target_tensor, center_index_target_tensor = getTargetIndexTensor(
            left_target_tensor_range, right_target_tensor_range, center_target_range)
        
        '''
        articulation target is size [seq, ]
        '''
        last_index = data_dim - 1
        articulate_y = music_data[part, left_range_start+1:left_range_end+1, last_index]
        articulate_target = torch.tensor(articulate_y).float()
        articulate_target = articulate_target.view(-1, 1)
        
        miniBatches.append((left_tensor, right_tensor, center_tensor, left_index_target_tensor, right_index_target_tensor, center_index_target_tensor, articulate_target))
        
        i += 1
        center_inx = i + seq_length
        
    return miniBatches
    

In [15]:
'''
mini generateMiniBatchFromOneMusicForPart test
'''
# fake_data = np.array([[
#         [1, 0, 0],
#         [0, 1, 0],
#         [0, 1, 0],
#         [1, 0, 0],
#         [0, 0, 1],
#         [0, 1, 0],
#         [0, 0, 1]],

#        [[1, 0, 0],
#         [1, 0, 0],
#         [0, 1, 0],
#         [0, 1, 0],
#         [1, 0, 0],
#         [0, 0, 1],
#         [0, 1, 0]],

#        [[1, 0, 0],
#         [0, 1, 0],
#         [0, 0, 1],
#         [0, 1, 0],
#         [0, 1, 0],
#         [1, 0, 0],
#         [1, 0, 0]],

#        [[0, 1, 0],
#         [0, 0, 1],
#         [0, 1, 0],
#         [0, 0, 1],
#         [1, 0, 0],
#         [0, 1, 0],
#         [0, 1, 0]]])
# # print('fake_data')
# # print(fake_data)

# r = generateMiniBatchFromOneMusicForPart(fake_data, 3, 0)
# lt, rt, ct, lt_t, rt_t, ct_t = r[0]

'\nmini generateMiniBatchFromOneMusicForPart test\n'

In [16]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [17]:
demo_piece_list = dataset.__getitem__(3) #numpy array
demo_piece = demo_piece_list[0, :, :, :]
print('demo_piece shape: {x}'.format(x=demo_piece.shape))
parts, ticks, data_dim = demo_piece.shape

'''
data_dim should be 73
'''
note_size = data_dim - 1

lstm_input_size = data_dim
lstm_hidden_size = 200
lstm_output_size = note_size
center_input_size = data_dim * 3
center_output_size = note_size

arti_lstm_hidden_size = 200
arti_output_size = 1
arti_center_input_size = 3

n_iters = 2000
print_every = 100
plot_every = 500
all_losses = []
learning_rate = 0.005

'''
(1) Data representation:
Each tick contains 73 entries.
0-70: notes one-hot representation
71: boolean rest representation
72: boolean articulation representation

(2) Model ensemble:
Four ensemble models. Each part has its own ensemble models.
Each ensemble model contains note prediction models + articulation prediction models

NOTE PREDICTION models:
left_LSTM. [vector input_size:  73 * seq_length. expected vector output_size: 72]
right_LSTM. [vector input_size:  73 * seq_length. expected vector output_size: 72]
harmony_Linear (center_linear) [vector input_size:  73 * 3. expected vector output_size: 72]
result_Linear [vector input_size:  72 * 3. expected vector output_size: 72]

ARTICULATION PREDICTION models:
left_LSTM. [vector input_size:  73. expected vector output_size: 1]
right_LSTM. [vector input_size:  73. expected vector output_size: 1]
result_Linear. [vector input_size:  2. expected vector output_size: 1]

The final prediction for each tick is the combination of the note prediction and the articulation prediction.
Final vector output_size: 72 + 1 = 73

'''

'''
PART 1
'''
lr_1_note = learning_rate
lr_1_arti = learning_rate

left_model = Left_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
right_model = Right_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
center_model = Center_Model(center_input_size, center_output_size)
result_model = Result_Model(center_output_size + lstm_output_size * 2, note_size)
arti_left_model = Arti_LSTM_Model(lstm_input_size, arti_lstm_hidden_size, arti_output_size)

# left_opt = optim.SGD(left_model.parameters(), lr=learning_rate)
# right_opt = optim.SGD(right_model.parameters(), lr=learning_rate)
# center_opt = optim.SGD(center_model.parameters(), lr=learning_rate)
result_opt = optim.SGD(result_model.parameters(), lr=lr_1_note)
arti_left_opt = optim.SGD(result_model.parameters(), lr=lr_1_arti)


'''
PART 2
'''
lr_2_note = learning_rate
lr_2_arti = learning_rate

left_model_2 = Left_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
right_model_2 = Right_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
center_model_2 = Center_Model(center_input_size, center_output_size)
result_model_2 = Result_Model(center_output_size + lstm_output_size * 2, note_size)
arti_left_model_2 = Arti_LSTM_Model(lstm_input_size, arti_lstm_hidden_size, arti_output_size)

result_opt_2 = optim.SGD(result_model.parameters(), lr=lr_2_note)
arti_left_opt_2 = optim.SGD(result_model.parameters(), lr=lr_2_arti)


'''
PART 3
'''
lr_3_note = learning_rate
lr_3_arti = learning_rate

left_model_3 = Left_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
right_model_3 = Right_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
center_model_3 = Center_Model(center_input_size, center_output_size)
result_model_3 = Result_Model(center_output_size + lstm_output_size * 2, note_size)
arti_left_model_3 = Arti_LSTM_Model(lstm_input_size, arti_lstm_hidden_size, arti_output_size)

result_opt_3 = optim.SGD(result_model.parameters(), lr=lr_3_note)
arti_left_opt_3 = optim.SGD(result_model.parameters(), lr=lr_3_arti)


'''
PART 4
'''
lr_4_note = learning_rate
lr_4_arti = learning_rate

left_model_4 = Left_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
right_model_4 = Right_Model(lstm_input_size, lstm_hidden_size, lstm_output_size)
center_model_4 = Center_Model(center_input_size, center_output_size)
result_model_4 = Result_Model(center_output_size + lstm_output_size * 2, note_size)
arti_left_model_4 = Arti_LSTM_Model(lstm_input_size, arti_lstm_hidden_size, arti_output_size)

result_opt_4 = optim.SGD(result_model.parameters(), lr=lr_4_note)
arti_left_opt_4 = optim.SGD(result_model.parameters(), lr=lr_4_arti)


demo_piece shape: (4, 1541, 73)


In [18]:
seq_length = 32
pt1_total_loss = 0 # Reset every plot_every iters
pt2_total_loss = 0 
pt3_total_loss = 0
pt4_total_loss = 0 


demo_piece_minibatch_pt1 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 0)
demo_piece_minibatch_pt2 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 1)
demo_piece_minibatch_pt3 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 2)
demo_piece_minibatch_pt4 = generateMiniBatchFromOneMusicForPart(demo_piece, seq_length, 3)

miniBatch_length = len(demo_piece_minibatch_pt1)
print("minibatch list length: {x}".format(x=miniBatch_length))


start = time.time()
ite = 0
'''
IMPORTANT: Model, Optimizer must match!
'''

'''
NOTE: This is only trainning for part 1, with data from one music piece.

'''
for n in range(n_iters):
    if ite/miniBatch_length >= 1:
        ite = 0
        
    '''
    PART 1
    '''
    left_tensor, right_tensor, center_tensor, left_target_tensor, right_targte_tensor, center_target_tensor, articulate_target = demo_piece_minibatch_pt1[ite]

    output, pt1_result_loss, arti_loss = train(left_model, right_model, center_model, result_model, arti_left_model,
          left_tensor, right_tensor, center_tensor,
           center_target_tensor, articulate_target, result_opt, arti_left_opt)
    pt1_total_loss += pt1_result_loss
    
    if (n+1) % print_every == 0:
            print('%s (%d %d%%) p1 loss: %.4f' % (timeSince(start), n, n / n_iters * 100, pt1_result_loss))
#             print("arti_left_opt: {x}".format(x=arti_left_opt))

    '''
    PART 2
    '''
    left_tensor, right_tensor, center_tensor, left_target_tensor, right_targte_tensor, center_target_tensor, articulate_target = demo_piece_minibatch_pt2[ite]

    output, pt2_result_loss, arti_loss = train(left_model_2, right_model_2, center_model_2, result_model_2, arti_left_model_2,
          left_tensor, right_tensor, center_tensor,
          center_target_tensor, articulate_target, result_opt_2, arti_left_opt_2)
    pt2_total_loss += pt2_result_loss
    
    if (n+1) % print_every == 0:
            print('%s (%d %d%%) p2 loss: %.4f' % (timeSince(start), n, n / n_iters * 100, pt2_result_loss))
            
            
    '''
    PART 3
    '''
    left_tensor, right_tensor, center_tensor, left_target_tensor, right_targte_tensor, center_target_tensor, articulate_target = demo_piece_minibatch_pt2[ite]

    output, pt3_result_loss, arti_loss = train(left_model_3, right_model_3, center_model_3, result_model_3, arti_left_model_3,
          left_tensor, right_tensor, center_tensor,
          center_target_tensor, articulate_target, result_opt_3, arti_left_opt_3)
    pt3_total_loss += pt3_result_loss
    
    if (n+1) % print_every == 0:
            print('%s (%d %d%%) p3 loss: %.4f' % (timeSince(start), n, n / n_iters * 100, pt3_result_loss))
            
            
    '''
    PART 4
    '''
    left_tensor, right_tensor, center_tensor, left_target_tensor, right_targte_tensor, center_target_tensor, articulate_target = demo_piece_minibatch_pt2[ite]

    output, pt4_result_loss, arti_loss = train(left_model_4, right_model_4, center_model_4, result_model_4, arti_left_model_4,
          left_tensor, right_tensor, center_tensor,
          center_target_tensor, articulate_target, result_opt_4, arti_left_opt_4)
    pt4_total_loss += pt4_result_loss
    
    if (n+1) % print_every == 0:
            print('%s (%d %d%%) p4 loss: %.4f' % (timeSince(start), n, n / n_iters * 100, pt4_result_loss))
            
  
    ite += 1
    n += 1
    
# print(n)


minibatch list length: 1477


/Users/qui-gon/Workspace/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/qui-gon/Workspace/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


0m 16s (99 4%) p1 loss: -0.0144
0m 16s (99 4%) p2 loss: -0.0136
0m 16s (99 4%) p3 loss: -0.0137
0m 16s (99 4%) p4 loss: -0.0136
0m 32s (199 9%) p1 loss: -0.0137
0m 32s (199 9%) p2 loss: -0.0136
0m 32s (199 9%) p3 loss: -0.0137
0m 32s (199 9%) p4 loss: -0.0136
0m 49s (299 14%) p1 loss: -0.0140
0m 49s (299 14%) p2 loss: -0.0136
0m 49s (299 14%) p3 loss: -0.0137
0m 49s (299 14%) p4 loss: -0.0145
1m 5s (399 19%) p1 loss: -0.0137
1m 5s (399 19%) p2 loss: -0.0139
1m 5s (399 19%) p3 loss: -0.0139
1m 5s (399 19%) p4 loss: -0.0136
1m 21s (499 24%) p1 loss: -0.0138
1m 21s (499 24%) p2 loss: -0.0148
1m 21s (499 24%) p3 loss: -0.0137
1m 21s (499 24%) p4 loss: -0.0136
1m 37s (599 29%) p1 loss: -0.0145
1m 37s (599 29%) p2 loss: -0.0136
1m 37s (599 29%) p3 loss: -0.0137
1m 37s (599 29%) p4 loss: -0.0136
1m 52s (699 34%) p1 loss: -0.0137
1m 52s (699 34%) p2 loss: -0.0136
1m 52s (699 34%) p3 loss: -0.0137
1m 52s (699 34%) p4 loss: -0.0136
2m 8s (799 39%) p1 loss: -0.0146
2m 8s (799 39%) p2 loss: -0.013

In [19]:
import numpy as np
from pdb import set_trace
from functools import reduce

# Number of times to repeat the sampling process, this is a multiplier of the
# number of "nodes".
# Example
#   Four parts and 128 ticks gives 512 nodes, num_repeats = 10 will run the
#       sampling process 5120 times.
num_repeats = 1


def sample(**kwargs):
    '''
    Give the number of ticks (each tick is the length of an eighth note), use the model to populate the music.
    '''
    global num_repeats
    num_parts = kwargs.get("num_parts", 4)
    # total number of time slices, default 8 measures
    num_ticks = kwargs.get("num_ticks", 128)
    num_dims = kwargs.get("num_dims", 73)
    # sequence length that the network takes in, default 2 measures
    seq_len = kwargs.get("seq_len", 32)
    part_nums = [0, 1, 2, 3]

    # models
    models = {}
    for part_num in part_nums:
        models["pitch_next_"+str(part_num)] = \
            kwargs.get("pitch_next_"+str(part_num), None)
        models["pitch_prev_"+str(part_num)] = \
            kwargs.get("pitch_prev_"+str(part_num), None)
        models["harmony_"+str(part_num)] = \
            kwargs.get("harmony_"+str(part_num), None)
        models["pitch_combined_"+str(part_num)] = \
            kwargs.get("pitch_combined_"+str(part_num), None)
        models["rhythm_next_"+str(part_num)] = \
            kwargs.get("rhythm_next_"+str(part_num), None)
        
        
    for key, val in models.items():
        val.eval()

    result = np.zeros((num_parts, num_ticks, num_dims))

    # initialize all of the parts
    for part_num in part_nums:
        result[part_num, :, :] = _populate_part(num_ticks, num_dims)

    # go back and revise it
    num_cells = reduce(lambda x, y: x*y, result.shape[:-1])
    # number of "nodes" to revise
    max_iters = num_cells * num_repeats
    # list of indices to revise
    part_indices = np.random.randint(0, num_parts, max_iters)
    tick_indices = np.random.randint(0+seq_len, num_ticks-seq_len, max_iters)
    for iter_i, (i_part, i_tick) in enumerate(zip(part_indices, tick_indices)):
        # prev_seq.shape = (seq_len, num_dims)
        prev_seq = result[i_part, i_tick-seq_len:i_tick, :]
        prev_seq = prev_seq.reshape((seq_len, 1, num_dims)).copy()
        prev_seq = torch.FloatTensor(prev_seq)

        # next_seq.shape = (seq_len, num_dims)
        next_seq = result[i_part, i_tick:i_tick+seq_len, :]
        next_seq = next_seq.reshape((seq_len, 1, num_dims)).copy()
        next_seq = torch.FloatTensor(prev_seq)

        # harmony.shape = (num_parts, num_dims)
        harmony = result[:, i_tick, :].copy()
        # harmony.shape = (num_parts - 1, num_dims)
        harmony = np.delete(harmony, i_part, axis=0)
        x, y = harmony.shape
        harmony = harmony.reshape(1, x*y)
        harmony = torch.FloatTensor(harmony)
        

        # getting outputs from pitch models
        pn_output, _ = models["pitch_next_"+str(i_part)](prev_seq)
        pn_output = pn_output[-1, :]
        assert pn_output.shape == (num_dims - 1,), \
            "Invalid pitch_next model output shape."
        pp_output, _ = models["pitch_prev_"+str(i_part)](next_seq)
        pp_output = pp_output[-1, :]
        assert pp_output.shape == (num_dims - 1,), \
            "Invalid pitch_prev model output shape."
        hm_output = models["harmony_"+str(i_part)](harmony)
        hm_output = hm_output.squeeze()
        assert hm_output.shape == (num_dims - 1,), \
            "Invalid harmony model output shape."
        # getting outputs from rhythm model
        rm_output, _ = models["rhythm_next_"+str(part_num)](prev_seq)
        rm_output = rm_output[-1]
        assert rm_output.shape == (1,), \
            "Invalid rhythm model output shape."

        # input for the pitch_combined model
        comb_input = torch.zeros((3, num_dims - 1)).float()
        comb_input[0, :] = pn_output
        comb_input[1, :] = pp_output
        comb_input[2, :] = hm_output
        x, y = comb_input.shape
        comb_input = comb_input.reshape(1, x*y)
        cb_output = models["pitch_combined_"+str(i_part)](comb_input)
        cb_output = cb_output.squeeze()
        assert cb_output.shape == (num_dims - 1,), \
            "Invalid combined output shape."

        # update the pitch based on the pitch_combined output
        result[i_part, i_tick, cb_output.argmax()] = 1
        # update the rhythm
        result[i_part, i_tick, -1] = rm_output
        
        if (iter_i % 100) == 0:
            print("current iter: {x} / {y}".format(x=iter_i, y=max_iters))

    return result

def _populate_part(num_ticks, num_dims):
    '''
    Initial run through the part to populate the part, with only pitch_next and rhythm_next layer.
    '''
    part = np.zeros((num_ticks, num_dims))
    pitches = np.random.randint(0, num_dims - 1, num_ticks)
    rhythm = np.random.randint(0, 2, num_ticks)
    part[np.arange(num_ticks), pitches] = 1
    part[np.arange(num_ticks), -1] = rhythm
    
    return part

In [20]:

result = sample(num_ticks=1024, seq_len=seq_length, 
                pitch_next_0=left_model, pitch_next_1=left_model_2, pitch_next_2=left_model_3, pitch_next_3=left_model_4,
                pitch_prev_0=right_model, pitch_prev_1=right_model_2, pitch_prev_2=right_model_3, pitch_prev_3=right_model_4,
                harmony_0=center_model, harmony_1=center_model_2, harmony_2=center_model_3, harmony_3=center_model_4,
                pitch_combined_0=result_model, pitch_combined_1=result_model_2, pitch_combined_2=result_model_3, pitch_combined_3=result_model_4,
                rhythm_next_0=arti_left_model, rhythm_next_1=arti_left_model_2, rhythm_next_2=arti_left_model_3, rhythm_next_3=arti_left_model_4)


/Users/qui-gon/Workspace/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/qui-gon/Workspace/anaconda3/envs/cs682project/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


current iter: 0 / 4096
current iter: 100 / 4096
current iter: 200 / 4096
current iter: 300 / 4096
current iter: 400 / 4096
current iter: 500 / 4096
current iter: 600 / 4096
current iter: 700 / 4096
current iter: 800 / 4096
current iter: 900 / 4096
current iter: 1000 / 4096
current iter: 1100 / 4096
current iter: 1200 / 4096
current iter: 1300 / 4096
current iter: 1400 / 4096
current iter: 1500 / 4096
current iter: 1600 / 4096
current iter: 1700 / 4096
current iter: 1800 / 4096
current iter: 1900 / 4096
current iter: 2000 / 4096
current iter: 2100 / 4096
current iter: 2200 / 4096
current iter: 2300 / 4096
current iter: 2400 / 4096
current iter: 2500 / 4096
current iter: 2600 / 4096
current iter: 2700 / 4096
current iter: 2800 / 4096
current iter: 2900 / 4096
current iter: 3000 / 4096
current iter: 3100 / 4096
current iter: 3200 / 4096
current iter: 3300 / 4096
current iter: 3400 / 4096
current iter: 3500 / 4096
current iter: 3600 / 4096
current iter: 3700 / 4096
current iter: 3800 / 409

In [35]:
print(result.shape)
music = dataset.matrix_to_score(result)

(4, 1024, 73)


In [36]:
import os
from music21 import converter
converter.freeze(music, fp=os.getcwd() + "/outputs/score_1.pgz")

'/Users/qui-gon/School/COMPSCI 682 Neural Networks/cs682-project/outputs/score_1.pgz'